In [1]:
# Use modules from parent directory
import os
import sys
import tifffile as tiff
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000   
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
from srcv2_2.models.params import get_params
from srcv2_2.models.Unet import Unet, UnetV2
from srcv2_2.visualization.visualization_utils import get_predicted_thumbnails 
from srcv2_2.utils import get_model_name, get_cls

import tensorflow as tf

2024-05-04 18:28:35.254417: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-04 18:28:35.272616: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 18:28:35.272628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 18:28:35.273184: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-04 18:28:35.276311: I tensorflow/core/platform/cpu_feature_guar

## Defining Params

For development purposes, use epochs=5. For final use, use epochs=20 perhaps.

In [3]:
# new CE test

params = get_params('U-net', 'Landsat8')
params.cls = ['clear', 'cloud', 'thin', 'shadow'] # get_cls(params.satellite, params.train_dataset, cls_string=['clear', 'cloud', 'thin', 'shadow'])  #'clear', 'cloud', 'shadow', 'snow', 'water'] 

#working edition: ['cloud', 'thin']
params.collapse_cls = False
params.project_path = "/home/mxh/RS-Net/"
params.activation_func='elu'
params.last_layer_activation_func = 'softmax'
params.loss_func="categorical_crossentropy"
params.learning_rate = 1e-6
params.epochs = 3
params.threshold = 0.5
params.dropout = 0.0 # 0.2
params.dropout_on_last_layer_only = True
params.L2reg = 1e-4
params.decay = 0.0
params.num_gpus = 1


params.overlap = 40
params.modelID = "CE-Test-1"
params.norm_threshold = 16000 # valid range -100 - 16000 in MOD09 doc

#tile = 'all'  # Name of one tile or 'all'
#tile = 'L2A_T32UNF_20170803T103019' 

In [2]:
# sparse ce test 
# sparse ce trains on only integers as classes. So convert cls to integer values defined by fmask/gt (from get_cls)

params = get_params('U-net-v2', 'Landsat8')
params.cls = get_cls(params.satellite, 'Biome_gt', cls_string=['fill', 'shadow', 'clear', 'thin', 'cloud' ])  #'clear', 'cloud', 'shadow', 'snow', 'water'] 
print(params.cls)
#working edition: ['cloud', 'thin']
params.collapse_cls = False
params.project_path = "/home/mxh/RS-Net/"
params.activation_func='elu'
params.last_layer_activation_func = 'softmax'
params.loss_func="sparse_categorical_crossentropy"
params.learning_rate = 1e-6
params.epochs = 5
params.threshold = 0.5
params.dropout = 0 # 0.2
params.dropout_on_last_layer_only = True
params.L2reg = 1e-4
params.decay = 0.2
params.num_gpus = 1


params.overlap = 40
params.modelID = "SparseCE-Test-2"
params.norm_threshold = 16000 # valid range -100 - 16000 in MOD09 doc

#tile = 'all'  # Name of one tile or 'all'
#tile = 'L2A_T32UNF_20170803T103019' 

[0, 64, 128, 192, 255]


## Training of local model

In [3]:
params.__dict__

{'modelNick': 'Unet-MOD09GA',
 'modelID': 'SparseCE-Test-2',
 'num_gpus': 1,
 'optimizer': 'Adam',
 'loss_func': 'sparse_categorical_crossentropy',
 'activation_func': 'elu',
 'last_layer_activation_func': 'softmax',
 'initialization': 'glorot_uniform',
 'use_batch_norm': True,
 'dropout_on_last_layer_only': True,
 'early_stopping': False,
 'reduce_lr': False,
 'save_best_only': False,
 'use_ensemble_learning': False,
 'ensemble_method': 'Bagging',
 'learning_rate': 1e-06,
 'dropout': 0,
 'L1reg': 0.0,
 'L2reg': 0.0001,
 'L1L2reg': 0.0,
 'decay': 0.2,
 'batch_norm_momentum': 0.7,
 'threshold': 0.5,
 'patch_size': 256,
 'overlap': 40,
 'overlap_train_set': 0,
 'batch_size': 40,
 'steps_per_epoch': None,
 'epochs': 5,
 'norm_method': 'enhance_contrast',
 'norm_threshold': 16000,
 'cls': [0, 64, 128, 192, 255],
 'collapse_cls': False,
 'affine_transformation': True,
 'brightness_augmentation': False,
 'bands': [1, 2, 3, 4, 5, 6, 7],
 'project_path': '/home/mxh/RS-Net/',
 'toa_path': 'data

In [4]:
# training local (notebook) model
local_model = UnetV2(params)
local_model.get_config()

No model was loaded.


2024-05-04 18:28:44.201108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-04 18:28:44.220728: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-04 18:28:44.220817: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

{'seed': 1,
 'params': <srcv2_2.models.params.HParams at 0x714918f61710>,
 'n_cls': 5,
 'n_bands': 7,
 'model_config': {'name': 'model',
  'trainable': True,
  'layers': [{'module': 'keras.layers',
    'class_name': 'InputLayer',
    'config': {'batch_input_shape': (None, 256, 256, 7),
     'dtype': 'float32',
     'sparse': False,
     'ragged': False,
     'name': 'input_1'},
    'registered_name': None,
    'name': 'input_1',
    'inbound_nodes': []},
   {'module': 'keras.layers',
    'class_name': 'Conv2D',
    'config': {'name': 'conv2d',
     'trainable': True,
     'dtype': 'float32',
     'filters': 32,
     'kernel_size': (3, 3),
     'strides': (1, 1),
     'padding': 'same',
     'data_format': 'channels_last',
     'dilation_rate': (1, 1),
     'groups': 1,
     'activation': 'elu',
     'use_bias': True,
     'kernel_initializer': {'module': 'keras.initializers',
      'class_name': 'GlorotUniform',
      'config': {'seed': None},
      'registered_name': None},
     'bias

In [5]:
hist = local_model.train()

Model: Training on cls categories:  [0, 64, 128, 192, 255]
Compiling with Sparse Categorical Crossentropy
------------------------------------------
Start training:
Epoch 1/5


2024-05-04 18:28:47.051677: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8800


477/477 [==============================] - 129s 236ms/step - loss: 1.8613 - sparse_categorical_crossentropy: 1.5770 - jaccard_coef_loss: 39.7801 - jaccard_coef: 1.1574e-16 - jaccard_coef_thresholded: 4.8986e-15 - sparse_categorical_accuracy: 0.2033 - val_loss: 1.7964 - val_sparse_categorical_crossentropy: 1.5121 - val_jaccard_coef_loss: 39.7802 - val_jaccard_coef: 1.1574e-16 - val_jaccard_coef_thresholded: 7.0000e-15 - val_sparse_categorical_accuracy: 0.2526
Epoch 2/5
477/477 [==============================] - 115s 239ms/step - loss: 1.7689 - sparse_categorical_crossentropy: 1.4846 - jaccard_coef_loss: 39.7801 - jaccard_coef: 1.1574e-16 - jaccard_coef_thresholded: 6.3033e-15 - sparse_categorical_accuracy: 0.2661 - val_loss: 1.7489 - val_sparse_categorical_crossentropy: 1.4645 - val_jaccard_coef_loss: 39.7802 - val_jaccard_coef: 1.1574e-16 - val_jaccard_coef_thresholded: 7.6550e-15 - val_sparse_categorical_accuracy: 0.2933
Epoch 3/5
477/477 [==============================] - 115s 239ms/

In [9]:

hist.params

{'verbose': 1, 'epochs': 5, 'steps': 477}

In [1]:
# check if training_params are saved
local_model.get_config()["params"].__dict__

NameError: name 'local_model' is not defined